**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Production economy and CO2 taxation](#toc1_)    
- 2. [Problem 2: Career choice model](#toc2_)    
- 3. [Problem 3: Barycentric interpolation](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
# Write your code here
import numpy as np
from types import SimpleNamespace

## 1. <a id='toc1_'></a>[Problem 1: Production economy and CO2 taxation](#toc0_)

Consider a production economy with two firms indexed by $j \in \{1,2\}$. Each produce its own good. They solve

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

Optimal firm behavior is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The implied profits are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\cdot\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single consumer supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm.<br>
She solves:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax and $T$ is lump-sum transfer. <br>
For a given $\ell$, it can be shown that optimal behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$
Such that optimal behavior is:
$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\cdot \left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$
With optimal consumption:
$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$


The government chooses $\tau$ and balances its budget so $T=\tau c_2^*$. We initially set $\tau,T=0$.

Market clearing requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$


**Question 1:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`. We choose $w=1$ as numeraire.

In [2]:
par = SimpleNamespace()

# firms
par.A = 1.0
par.gamma = 0.5

# households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0

# government
par.tau = 0.0
par.T = 0.0

# Question 3
par.kappa = 0.1

In [3]:
import numpy as np
from scipy.optimize import minimize

class EconomicModel:
    def __init__(self, alpha, nu, epsilon, A, gamma):
        self.alpha = alpha
        self.nu = nu
        self.epsilon = epsilon
        self.A = A
        self.gamma = gamma
    
    def optimal_labor(self, p1, p2, w, tau, T):
        # Calculate optimal labor
        return ((p1 * self.A * self.gamma) / w) ** (1 / self.gamma), ((p2 * self.A * self.gamma) / w) ** (1 / self.gamma)

    def optimal_profits(self, p, w):
        # Calculate optimal profits
        return (1 - self.gamma) / self.gamma * w * (p * self.A * self.gamma / w) ** (1 - self.gamma)

    def consumption(self, w, T, p1, p2, tau):
        # Calculate consumption
        labor1, labor2 = self.optimal_labor(p1, p2, w, tau, T)
        profits1 = self.optimal_profits(p1, w)
        profits2 = self.optimal_profits(p2, w)
        c1 = self.alpha * (w * (labor1 + labor2) + T + profits1 + profits2) / p1
        c2 = (1 - self.alpha) * (w * (labor1 + labor2) + T + profits1 + profits2) / (p2 + tau)
        return c1, c2
    
    def utility(self, c1, c2, l):
        # Calculate utility
        return self.alpha * np.log(c1) + (1 - self.alpha) * np.log(c2) - self.nu * l ** (1 + self.epsilon) / (1 + self.epsilon)
    
    def labor_demand(self, p1, p2, w):
        # Calculate labor demand for both goods
        L1 = (p1 * self.A * self.gamma / w) ** (1 / (1 - self.gamma))
        L2 = (p2 * self.A * self.gamma / w) ** (1 / (1 - self.gamma))
        return L1, L2

    def production(self, labor):
        # Production function based on labor
        return self.A * labor ** self.gamma

    def check_market_clearing(self, p1, p2, w, tau, T):
        """Check market clearing conditions for labor and goods markets."""
        labor_demand1, labor_demand2 = self.labor_demand(p1, p2, w)
        production1 = self.production(labor_demand1)
        production2 = self.production(labor_demand2)
        consumption1, consumption2 = self.consumption(w, T, p1, p2, tau)
        
        return {
            'Labor Market': np.isclose(labor_demand1 + labor_demand2, labor_demand1 + labor_demand2),
            'Good Market 1': np.isclose(production1, consumption1),
            'Good Market 2': np.isclose(production2, consumption2)
        }

    def solve_equilibrium(self, w, tau, T):
        """
        Solve for the equilibrium prices p1 and p2 that clear the markets.
        """

        def market_excess_demand(prices):
            p1, p2 = prices
            labor_demand1, labor_demand2 = self.labor_demand(p1, p2, w)
            production1 = self.production(labor_demand1)
            production2 = self.production(labor_demand2)
            consumption1, consumption2 = self.consumption(w, T, p1, p2, tau)
            excess_demand1 = production1 - consumption1
            excess_demand2 = production2 - consumption2
            return excess_demand1**2 + excess_demand2**2

        result = minimize(market_excess_demand, [1.0, 1.0], bounds=[(0.1, 2.0), (0.1, 2.0)])
        if result.success:
            return result.x
        else:
            raise ValueError("Equilibrium prices could not be found")

# Example usage:
alpha = 0.3
nu = 1.0
epsilon = 1.5
A = 1.0
gamma = 0.3
w = 1.0
tau = 0.1
T = 0.5

model = EconomicModel(alpha, nu, epsilon, A, gamma)
p1, p2 = model.solve_equilibrium(w, tau, T)
print(f"p1={p1:.2f} and p2={p2:.2f}")


p1=1.31 and p2=2.00


In [4]:
import numpy as np
from types import SimpleNamespace

# Parameters
par = SimpleNamespace()

# Firms
par.A = 1.0
par.gamma = 0.5

# Households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0

# Government
par.tau = 0.0
par.T = 0.0

# Question 3
par.kappa = 0.1

# Firm's optimal labor and output
def firm_behavior(w, p, A, gamma):
    L_star = (p * A * gamma / w) ** (1 / (1 - gamma))
    Y_star = A * L_star ** gamma
    Pi_star = w * L_star * (1 - gamma) / gamma
    return L_star, Y_star, Pi_star

# Consumer's optimal consumption
def consumer_behavior(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2):
    L_star = (alpha / p1 + (1 - alpha) / (p2 + tau)) * (w + T + Pi1 + Pi2) ** (1 / (1 + epsilon)) / (nu ** (1 / (1 + epsilon)))
    C1_star = alpha * (w * L_star + T + Pi1 + Pi2) / p1
    C2_star = (1 - alpha) * (w * L_star + T + Pi1 + Pi2) / (p2 + tau)
    return L_star, C1_star, C2_star

# Market clearing
def market_clearing(w, p1, p2, tau, T, par):
    # Firm 1
    L1_star, Y1_star, Pi1_star = firm_behavior(w, p1, par.A, par.gamma)
    
    # Firm 2
    L2_star, Y2_star, Pi2_star = firm_behavior(w, p2, par.A, par.gamma)
    
    # Consumer
    L_star, C1_star, C2_star = consumer_behavior(w, p1, p2, tau, T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)
    
    # Market clearing conditions
    labor_market = L_star - L1_star - L2_star
    goods_market_1 = C1_star - Y1_star
    goods_market_2 = C2_star - Y2_star
    
    return labor_market, goods_market_1, goods_market_2

# Check market clearing for different prices
w = 1
p1_values = np.linspace(0.1, 2.0, 10)
p2_values = np.linspace(0.1, 2.0, 10)

market_clearing_results = []
for p1 in p1_values:
    for p2 in p2_values:
        market_clearing_results.append((p1, p2, market_clearing(w, p1, p2, par.tau, par.T, par)))

print(market_clearing_results)


[(0.1, 0.1, (10.01163896579312, 30.01491689737935, 70.10147276055181)), (0.1, 0.3111111111111111, (5.269613431672632, 15.919025480203086, 11.821213554596758)), (0.1, 0.5222222222222223, (4.3696866888891135, 13.483134140741415, 5.785608398581862)), (0.1, 0.7333333333333333, (3.9904532739340683, 12.743026488468871, 3.70384176148252)), (0.1, 0.9444444444444444, (3.778055990374125, 12.637130934085338, 2.6622454203165082)), (0.1, 1.1555555555555557, (3.635303550287281, 12.873873613824802, 2.031850548090692)), (0.1, 1.3666666666666667, (3.5235284229198784, 13.337251935426302, 1.6022950458857912)), (0.1, 1.577777777777778, (3.4236270796663426, 13.969955313073106, 1.284484784171218)), (0.1, 1.788888888888889, (3.3247212935687376, 14.739349065891398, 1.034601085889216)), (0.1, 2.0, (3.2199934246884947, 15.624980274065486, 0.8287476986409732)), (0.3111111111111111, 0.1, (8.00784222863045, 7.617780403004758, 56.378661032511914)), (0.3111111111111111, 0.3111111111111111, (3.2169278349222536, 3.039

In [5]:
import numpy as np
from types import SimpleNamespace
from scipy.optimize import fsolve

# Parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.kappa = 0.1



# Firm's optimal labor and output
def firm_behavior(w, p, A, gamma):
    L_star = (p * A * gamma / w) ** (1 / (1 - gamma))
    Y_star = A * L_star ** gamma
    Pi_star = w * L_star * (1 - gamma) / gamma
    return L_star, Y_star, Pi_star

# Consumer's optimal consumption
def consumer_behavior(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2):
    L_star = (alpha / p1 + (1 - alpha) / (p2 + tau)) * (w + T + Pi1 + Pi2) ** (1 / (1 + epsilon)) / (nu ** (1 / (1 + epsilon)))
    C1_star = alpha * (w * L_star + T + Pi1 + Pi2) / p1
    C2_star = (1 - alpha) * (w * L_star + T + Pi1 + Pi2) / (p2 + tau)
    return L_star, C1_star, C2_star

# Market clearing equations
def market_clearing_eqs(prices, w, tau, T, par):
    p1, p2 = prices
    
    # Firm 1
    L1_star, Y1_star, Pi1_star = firm_behavior(w, p1, par.A, par.gamma)
    
    # Firm 2
    L2_star, Y2_star, Pi2_star = firm_behavior(w, p2, par.A, par.gamma)
    
    # Consumer
    L_star, C1_star, C2_star = consumer_behavior(w, p1, p2, tau, T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)
    
    # Market clearing conditions
    labor_market = L_star - L1_star - L2_star
    goods_market_1 = C1_star - Y1_star
    goods_market_2 = C2_star - Y2_star
    
    return [goods_market_1, goods_market_2]

# Solve for equilibrium prices
w = 1
initial_guess = [1.0, 1.0]  # Initial guess for p1 and p2
equilibrium_prices = fsolve(market_clearing_eqs, initial_guess, args=(w, par.tau, par.T, par))

p1_eq, p2_eq = equilibrium_prices
print(f"Equilibrium prices: p1 = {p1_eq:.4f}, p2 = {p2_eq:.4f}")


Equilibrium prices: p1 = 1.0408, p2 = 1.5899


In [6]:
import numpy as np
from types import SimpleNamespace
import sympy as sp

# Parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.kappa = 0.1

class Model:
    def __init__(self, par, w, p1, p2):
        self.par = par
        self.w = w
        self.p1 = p1
        self.p2 = p2
        self.alpha = par.alpha
        self.nu = par.nu
        self.epsilon = par.epsilon
        self.tau = par.tau
        self.T = par.T
        self.gamma = par.gamma
        self.A = par.A

    def optimal_behavior(self, l):
        c1 = self.c1(l)
        c2 = self.c2(l)
        utility = np.log(c1**self.alpha * c2**(1 - self.alpha)) - self.nu * (l**(1 + self.epsilon)) / (1 + self.epsilon)
        return utility

    def c1(self, l):
        return self.alpha * (self.w * l + self.T + self.pi1() + self.pi2()) / self.p1

    def c2(self, l):
        return (1 - self.alpha) * (self.w * l + self.T + self.pi1() + self.pi2()) / (self.p2 + self.tau)

    def pi1(self):
        return (1 - self.gamma) / self.gamma * self.w * ((self.p1 * self.A * self.gamma / self.w)**(1 / (1 - self.gamma)))

    def pi2(self):
        return (1 - self.gamma) / self.gamma * self.w * ((self.p2 * self.A * self.gamma / self.w)**(1 / (1 - self.gamma)))

    def labor_market(self):
        l1_star = (self.p1 * self.A * self.gamma / self.w)**(1 / (1 - self.gamma))
        l2_star = (self.p2 * self.A * self.gamma / self.w)**(1 / (1 - self.gamma))
        return l1_star + l2_star

    def good_market_1(self):
        return self.c1_star() - self.y1_star()

    def good_market_2(self):
        return self.c2_star() - self.y2_star()

    def y1_star(self):
        return self.A * (self.labor_market() / 2)**self.gamma

    def y2_star(self):
        return self.A * (self.labor_market() / 2)**self.gamma

    def c1_star(self):
        return self.c1(self.labor_market())

    def c2_star(self):
        return self.c2(self.labor_market())

    def solve_L_star(self):
        # Define symbolic variables
        l = sp.symbols('l')

        # Define symbolic versions of pi1 and pi2
        pi1_sym = (1 - self.gamma) / self.gamma * self.w * ((self.p1 * self.A * self.gamma / self.w)**(1 / (1 - self.gamma)))
        pi2_sym = (1 - self.gamma) / self.gamma * self.w * ((self.p2 * self.A * self.gamma / self.w)**(1 / (1 - self.gamma)))

        # Define c1 and c2 in terms of l
        c1 = self.alpha * (self.w * l + self.T + pi1_sym + pi2_sym) / self.p1
        c2 = (1 - self.alpha) * (self.w * l + self.T + pi1_sym + pi2_sym) / (self.p2 + self.tau)

        # Define the utility function
        utility = sp.log(c1**self.alpha * c2**(1 - self.alpha)) - self.nu * (l**(1 + self.epsilon)) / (1 + self.epsilon)

        # Derive the first-order condition by differentiating the utility function with respect to l
        foc = sp.diff(utility, l)

        # Solve for l (L_star)
        L_star_solutions = sp.solve(foc, l)

        # Filter only real and positive solutions
        L_star = [sol for sol in L_star_solutions if sol.is_real and sol > 0]

        return L_star

# Example usage:
model = Model(par, w=1, p1=1, p2=1)
L_star_solutions = model.solve_L_star()

# Print all L_star solutions
for sol in L_star_solutions:
    print(f"L_star: {sol}")


L_star: 0.858094329496553


In [7]:
import numpy as np
from types import SimpleNamespace
from scipy.optimize import fsolve

# Parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.kappa = 0.1



# Firm's optimal labor and output
def firm_behavior(w, p, A, gamma):
    L_star = (p * A * gamma / w) ** (1 / (1 - gamma))
    Y_star = A * L_star ** gamma
    Pi_star = w * L_star * (1 - gamma) / gamma
    return L_star, Y_star, Pi_star

def solve_L_star(self):
        # Define symbolic variables
        l = sp.symbols('l')

        # Define symbolic versions of pi1 and pi2
        pi1_sym = (1 - self.gamma) / self.gamma * self.w * ((self.p1 * self.A * self.gamma / self.w)**(1 / (1 - self.gamma)))
        pi2_sym = (1 - self.gamma) / self.gamma * self.w * ((self.p2 * self.A * self.gamma / self.w)**(1 / (1 - self.gamma)))

        # Define c1 and c2 in terms of l
        c1 = self.alpha * (self.w * l + self.T + pi1_sym + pi2_sym) / self.p1
        c2 = (1 - self.alpha) * (self.w * l + self.T + pi1_sym + pi2_sym) / (self.p2 + self.tau)

        # Define the utility function
        utility = sp.log(c1**self.alpha * c2**(1 - self.alpha)) - self.nu * (l**(1 + self.epsilon)) / (1 + self.epsilon)

        # Derive the first-order condition by differentiating the utility function with respect to l
        foc = sp.diff(utility, l)

        # Solve for l (L_star)
        L_star_solutions = sp.solve(foc, l)

        # Filter only real and positive solutions
        L_star = [sol for sol in L_star_solutions if sol.is_real and sol > 0]

        return L_star

# Consumer's optimal consumption
def consumer_behavior(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2):
    C1_star = alpha * (w * L_star + T + Pi1 + Pi2) / p1
    C2_star = (1 - alpha) * (w * L_star + T + Pi1 + Pi2) / (p2 + tau)
    return L_star, C1_star, C2_star

# Market clearing equations
def market_clearing_eqs(prices, w, tau, T, par):
    p1, p2 = prices
    
    # Firm 1
    L1_star, Y1_star, Pi1_star = firm_behavior(w, p1, par.A, par.gamma)
    
    # Firm 2
    L2_star, Y2_star, Pi2_star = firm_behavior(w, p2, par.A, par.gamma)
    
    # Consumer
    L_star, C1_star, C2_star = consumer_behavior(w, p1, p2, tau, T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)
    
    # Market clearing conditions
    labor_market = L_star - L1_star - L2_star
    goods_market_1 = C1_star - Y1_star
    goods_market_2 = C2_star - Y2_star
    
    return [goods_market_1, goods_market_2]

# Solve for equilibrium prices
w = 1
initial_guess = [1.0, 1.0]  # Initial guess for p1 and p2
equilibrium_prices = fsolve(market_clearing_eqs, initial_guess, args=(w, par.tau, par.T, par))

p1_eq, p2_eq = equilibrium_prices
print(f"Equilibrium prices: p1 = {p1_eq:.4f}, p2 = {p2_eq:.4f}")


NameError: name 'L_star' is not defined

In [ ]:
import numpy as np
from types import SimpleNamespace
from scipy.optimize import fsolve
import sympy as sp

# Parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.kappa = 0.1

# Firm's optimal labor and output
def firm_behavior(w, p, A, gamma):
    L_star = (p * A * gamma / w) ** (1 / (1 - gamma))
    Y_star = A * L_star ** gamma
    Pi_star = w * L_star * (1 - gamma) / gamma
    return L_star, Y_star, Pi_star

# Symbolic solution for L_star
def solve_L_star(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2):
    l = sp.symbols('l')

    # Define c1 and c2 in terms of l
    c1 = alpha * (w * l + T + Pi1 + Pi2) / p1
    c2 = (1 - alpha) * (w * l + T + Pi1 + Pi2) / (p2 + tau)

    # Define the utility function
    utility = sp.log(c1**alpha * c2**(1 - alpha)) - nu * (l**(1 + epsilon)) / (1 + epsilon)

    # Derive the first-order condition by differentiating the utility function with respect to l
    foc = sp.diff(utility, l)

    # Solve for l (L_star)
    L_star_solutions = sp.solve(foc, l)

    # Filter only real and positive solutions
    L_star = [sol for sol in L_star_solutions if sol.is_real and sol > 0]
    return L_star[0] if L_star else None

# Consumer's optimal consumption
def consumer_behavior(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2):
    L_star = solve_L_star(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2)
    C1_star = alpha * (w * L_star + T + Pi1 + Pi2) / p1
    C2_star = (1 - alpha) * (w * L_star + T + Pi1 + Pi2) / (p2 + tau)
    return L_star, C1_star, C2_star

# Market clearing equations
def market_clearing_eqs(prices, w, tau, T, par):
    p1, p2 = prices

    # Firm 1
    L1_star, Y1_star, Pi1_star = firm_behavior(w, p1, par.A, par.gamma)
    
    # Firm 2
    L2_star, Y2_star, Pi2_star = firm_behavior(w, p2, par.A, par.gamma)
    
    # Consumer
    L_star, C1_star, C2_star = consumer_behavior(w, p1, p2, tau, T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)
    
    # Market clearing conditions
    labor_market = L_star - L1_star - L2_star
    goods_market_1 = C1_star - Y1_star
    goods_market_2 = C2_star - Y2_star
    
    return [goods_market_1, goods_market_2]

# Solve for equilibrium prices
w = 1
initial_guess = [1.0, 1.0]  # Initial guess for p1 and p2
equilibrium_prices = fsolve(market_clearing_eqs, initial_guess, args=(w, par.tau, par.T, par))

p1_eq, p2_eq = equilibrium_prices
print(f"Equilibrium prices: p1 = {p1_eq:.4f}, p2 = {p2_eq:.4f}")

# Calculate L_star under consumer_behavior using equilibrium prices
L1_star, Y1_star, Pi1_star = firm_behavior(w, p1_eq, par.A, par.gamma)
L2_star, Y2_star, Pi2_star = firm_behavior(w, p2_eq, par.A, par.gamma)
L_star, C1_star, C2_star = consumer_behavior(w, p1_eq, p2_eq, par.tau, par.T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)
print(f"L_star: {L_star:.4f}")
print(f"C1_star: {C1_star:.4f}")
print(f"C2_star: {C2_star:.4f}")


Equilibrium prices: p1 = 0.9759, p2 = 1.4908
L_star: 0.7937
C1_star: 0.4880
C2_star: 0.7454


In [ ]:
# Function to check market clearing conditions with given prices
def check_market_clearing_conditions(p1, p2, w, tau, T, par):
    # Firm 1
    L1_star, Y1_star, Pi1_star = firm_behavior(w, p1, par.A, par.gamma)
    
    # Firm 2
    L2_star, Y2_star, Pi2_star = firm_behavior(w, p2, par.A, par.gamma)
    
    # Consumer
    L_star, C1_star, C2_star = consumer_behavior(w, p1, p2, tau, T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)
    
    # Market clearing conditions
    labor_market = float(L_star - L1_star - L2_star)
    goods_market_1 = float(C1_star - Y1_star)
    goods_market_2 = float(C2_star - Y2_star)
    
    return labor_market, goods_market_1, goods_market_2

# Given equilibrium prices
p1_given = 1
p2_given = 1.00

# Check market clearing conditions with the given prices
labor_market, goods_market_1, goods_market_2 = check_market_clearing_conditions(p1_given, p2_given, w, par.tau, par.T, par)

print(f"Labor market condition: {labor_market:.4e}")
print(f"Goods market 1 condition: {goods_market_1:.4e}")
print(f"Goods market 2 condition: {goods_market_2:.4e}")

# Check if markets clear
if np.isclose(labor_market, 0, atol=1e-6) and np.isclose(goods_market_1, 0, atol=1e-6) and np.isclose(goods_market_2, 0, atol=1e-6):
    print("The given prices clear the market.")
else:
    print("The given prices do not clear the market.")


Labor market condition: 3.5809e-01
Goods market 1 condition: -9.2572e-02
Goods market 2 condition: 4.5067e-01
The given prices do not clear the market.


### Derivation of \( L^* \)

1. **Consumer's utility maximization problem**:
   The consumer maximizes their utility:
   $$
   U(c_1, c_2, \ell) = \log(c_1^\alpha c_2^{1-\alpha}) - \nu \frac{\ell^{1+\epsilon}}{1+\epsilon}
   $$
   subject to the budget constraint:
   $$
   p_1 c_1 + (p_2 + \tau) c_2 = w \ell + T + \pi_1 + \pi_2
   $$
   where \( \pi_1 \) and \( \pi_2 \) are the profits from firms.

2. **Optimal consumption**:
   The optimal consumption of goods \( c_1 \) and \( c_2 \) derived from the utility maximization problem are:
   $$
   c_1 = \alpha \frac{w \ell + T + \pi_1 + \pi_2}{p_1}
   $$
   $$
   c_2 = (1 - \alpha) \frac{w \ell + T + \pi_1 + \pi_2}{p_2 + \tau}
   $$

3. **Utility function substitution**:
   Substitute the optimal consumption back into the utility function to derive the labor supply:
   $$
   U = \log \left( \left( \alpha \frac{w \ell + T + \pi_1 + \pi_2}{p_1} \right)^\alpha \left( (1 - \alpha) \frac{w \ell + T + \pi_1 + \pi_2}{p_2 + \tau} \right)^{1 - \alpha} \right) - \nu \frac{\ell^{1+\epsilon}}{1+\epsilon}
   $$

4. **Simplify the log term**:
   $$
   U = \alpha \log \left( \alpha \frac{w \ell + T + \pi_1 + \pi_2}{p_1} \right) + (1 - \alpha) \log \left( (1 - \alpha) \frac{w \ell + T + \pi_1 + \pi_2}{p_2 + \tau} \right) - \nu \frac{\ell^{1+\epsilon}}{1+\epsilon}
   $$

5. **First-order condition**:
   Differentiate the utility function with respect to \( \ell \) and set it to zero to find the optimal labor supply:
   $$
   \frac{dU}{d\ell} = \frac{\alpha}{w \ell + T + \pi_1 + \pi_2} \cdot w \cdot \frac{\alpha}{p_1} + \frac{(1 - \alpha)}{w \ell + T + \pi_1 + \pi_2} \cdot w \cdot \frac{1 - \alpha}{p_2 + \tau} - \nu \ell^\epsilon = 0
   $$

6. **Simplify the first-order condition**:
   $$
   \frac{\alpha w}{p_1 (w \ell + T + \pi_1 + \pi_2)} + \frac{(1 - \alpha) w}{(p_2 + \tau)(w \ell + T + \pi_1 + \pi_2)} - \nu \ell^\epsilon = 0
   $$

7. **Combine terms and solve for \(\ell\)**:
   $$
   \frac{w}{w \ell + T + \pi_1 + \pi_2} \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) = \nu \ell^\epsilon
   $$
   $$
   \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) \frac{w}{w \ell + T + \pi_1 + \pi_2} = \nu \ell^\epsilon
   $$

8. **Isolate \(\ell\)**:
   $$
   \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) w = \nu \ell^\epsilon (w \ell + T + \pi_1 + \pi_2)
   $$
   $$
   \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) w = \nu \ell^\epsilon \cdot w \ell + \nu \ell^\epsilon (T + \pi_1 + \pi_2)
   $$

9. **Simplify and solve for \( L^* \)**:
   $$
   \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) \cdot (w + T + \pi_1 + \pi_2)^{\frac{1}{1 + \epsilon}} = \nu^{\frac{1}{1 + \epsilon}}
   $$
   $$
   L^* = \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) \left( w + T + \pi_1 + \pi_2 \right)^{\frac{1}{1 + \epsilon}} \left( \nu^{\frac{1}{1 + \epsilon}} \right)^{-1}
   $$
   $$
   L^* = \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) \left( w + T + \pi_1 + \pi_2 \ \right)^{\frac{1}{1 + \epsilon}} \left( \nu^{- \frac{1}{1 + \epsilon}} \right)
   $$
   $$
   L^* = \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) \left( w + T + \pi_1 + \pi_2 \right)^{\frac{1}{1 + \epsilon}} \cdot \frac{1}{\nu^{\frac{1}{1 + \epsilon}}}
   $$
   $$
   L^* = \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) \left( w + T + \pi_1 + \pi_2 \right)^{\frac{1}{1 + \epsilon}} \cdot \nu^{- \frac{1}{1 + \epsilon}}
   $$

Thus, the optimal labor supply \( L^* \) is given by:
$$
L^* = \left( \frac{\alpha}{p_1} + \frac{1 - \alpha}{p_2 + \tau} \right) \left( w + T + \pi_1 + \pi_2 \right)^{\frac{1}{1 + \epsilon}} \cdot \nu^{- \frac{1}{1 + \epsilon}}
$$


In [ ]:
import numpy as np
from types import SimpleNamespace
from scipy.optimize import fsolve

# Parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.kappa = 0.1

# Define the model
class Model:
    def __init__(self, par):
        self.par = par
    
    def optimal_behavior(self, l):
        c1 = self.c1(l)
        c2 = self.c2(l)
        utility = np.log(c1**self.par.alpha * c2**(1 - self.par.alpha)) - self.par.nu * (l**(1 + self.par.epsilon)) / (1 + self.par.epsilon)
        return utility

    def c1(self, l):
        return self.par.alpha * (self.w * l + self.par.T + self.pi1() + self.pi2()) / self.p1

    def c2(self, l):
        return (1 - self.par.alpha) * (self.w * l + self.par.T + self.pi1() + self.pi2()) / (self.p2 + self.par.tau)

    def pi1(self):
        return (1 - self.par.gamma) / self.par.gamma * self.w * ((self.p1 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma)))

    def pi2(self):
        return (1 - self.par.gamma) / self.par.gamma * self.w * ((self.p2 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma)))

    def labor_market(self):
        l1_star = (self.p1 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma))
        l2_star = (self.p2 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma))
        return l1_star + l2_star

    def good_market_1(self):
        return self.c1_star() - self.y1_star()

    def good_market_2(self):
        return self.c2_star() - self.y2_star()

    def y1_star(self):
        return self.par.A * (self.labor_market() / 2)**self.par.gamma

    def y2_star(self):
        return self.par.A * (self.labor_market() / 2)**self.par.gamma

    def c1_star(self):
        return self.c1(self.labor_market())

    def c2_star(self):
        return self.c2(self.labor_market())

# Firm's optimal labor and output
def firm_behavior(w, p, A, gamma):
    L_star = (p * A * gamma / w) ** (1 / (1 - gamma))
    Y_star = A * L_star ** gamma
    Pi_star = w * L_star * (1 - gamma) / gamma
    return L_star, Y_star, Pi_star

# Consumer's optimal consumption
def consumer_behavior(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2):
    L_star = (alpha / p1 + (1 - alpha) / (p2 + tau)) * (w + T + Pi1 + Pi2) ** (1 / (1 + epsilon)) / (nu ** (1 / (1 + epsilon)))
    C1_star = alpha * (w * L_star + T + Pi1 + Pi2) / p1
    C2_star = (1 - alpha) * (w * L_star + T + Pi1 + Pi2) / (p2 + tau)
    return L_star, C1_star, C2_star

# Market clearing equations
def market_clearing_eqs(prices, w, tau, T, par):
    p1, p2 = prices
    
    # Firm 1
    L1_star, Y1_star, Pi1_star = firm_behavior(w, p1, par.A, par.gamma)
    
    # Firm 2
    L2_star, Y2_star, Pi2_star = firm_behavior(w, p2, par.A, par.gamma)
    
    # Consumer
    L_star, C1_star, C2_star = consumer_behavior(w, p1, p2, tau, T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)
    
    # Market clearing conditions
    labor_market = L_star - L1_star - L2_star
    goods_market_1 = C1_star - Y1_star
    goods_market_2 = C2_star - Y2_star
    
    return [goods_market_1, goods_market_2]

# Solve for equilibrium prices
w = 1
initial_guess = [1.0, 1.0]  # Initial guess for p1 and p2
equilibrium_prices = fsolve(market_clearing_eqs, initial_guess, args=(w, par.tau, par.T, par))

p1_eq, p2_eq = equilibrium_prices
print(f"Equilibrium prices: p1 = {p1_eq:.4f}, p2 = {p2_eq:.4f}")

Equilibrium prices: p1 = 1.0408, p2 = 1.5899


In [ ]:
import numpy as np
from types import SimpleNamespace
from scipy.optimize import minimize
import sympy as sp

# Parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.kappa = 0.1

# Firm's optimal labor and output
def firm_behavior(w, p, A, gamma):
    L_star = max((p * A * gamma / w) ** (1 / (1 - gamma)), 0)
    Y_star = A * L_star ** gamma
    Pi_star = max(w * L_star * (1 - gamma) / gamma, 0)
    return L_star, Y_star, Pi_star

# Derive labor supply L_star
def derive_labor_supply(par, w, p1, p2, tau, T, Pi1, Pi2):
    l = sp.symbols('l')
    
    # Consumption functions in terms of labor supply l
    c1 = par.alpha * (w * l + T + Pi1 + Pi2) / p1
    c2 = (1 - par.alpha) * (w * l + T + Pi1 + Pi2) / (p2 + tau)
    
    # Utility function
    utility = sp.log(c1**par.alpha * c2**(1 - par.alpha)) - par.nu * (l**(1 + par.epsilon)) / (1 + par.epsilon)
    
    # First-order condition (FOC) for utility maximization
    foc = sp.diff(utility, l)
    
    # Solve the FOC for labor supply L_star
    L_star_solutions = sp.solve(foc, l)
    
    # Filter only real and positive solutions
    L_star = [sol.evalf() for sol in L_star_solutions if sol.is_real and sol > 0]
    
    return L_star[0] if L_star else None

# Consumer's optimal consumption
def consumer_behavior(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2):
    # Derive L_star
    L_star = derive_labor_supply(par, w, p1, p2, tau, T, Pi1, Pi2)
    
    # Consumption given L_star
    C1_star = alpha * (w * L_star + T + Pi1 + Pi2) / p1
    C2_star = (1 - alpha) * (w * L_star + T + Pi1 + Pi2) / (p2 + tau)
    
    return L_star, C1_star, C2_star

# Market clearing equations
def market_clearing_eqs(prices, w, tau, T, par):
    p1, p2 = prices
    
    # Firm 1
    L1_star, Y1_star, Pi1_star = firm_behavior(w, p1, par.A, par.gamma)
    
    # Firm 2
    L2_star, Y2_star, Pi2_star = firm_behavior(w, p2, par.A, par.gamma)
    
    # Consumer
    L_star, C1_star, C2_star = consumer_behavior(w, p1, p2, tau, T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)
    
    # Market clearing conditions
    labor_market = L_star - (L1_star + L2_star)
    goods_market_1 = C1_star - Y1_star
    goods_market_2 = C2_star - Y2_star
    
    return labor_market, goods_market_1, goods_market_2

# Function to check market clearing conditions
def check_market_clearing(p1, p2, w, par):
    tau, T = par.tau, par.T
    
    labor_market, goods_market_1, goods_market_2 = market_clearing_eqs([p1, p2], w, tau, T, par)
    
    print(f"Labor Market Clearing Condition: {labor_market:.6f}")
    print(f"Goods Market 1 Clearing Condition: {goods_market_1:.6f}")
    print(f"Goods Market 2 Clearing Condition: {goods_market_2:.6f}")

# Objective function for minimization (sum of squared market clearing conditions)
def objective(prices, w, tau, T, par):
    conditions = market_clearing_eqs(prices, w, tau, T, par)
    return sum(cond**2 for cond in conditions)

# Constraints to ensure prices are non-negative
def price_constraint(prices):
    return prices

# Solve for equilibrium prices using minimize
w = 1
initial_guess = [1.0, 1.0]  # Initial guess for p1 and p2
bounds = [(0, None), (0, None)]  # Prices must be non-negative
constraints = {'type': 'ineq', 'fun': price_constraint}

result = minimize(objective, initial_guess, args=(w, par.tau, par.T, par), bounds=bounds, constraints=constraints)
p1_eq, p2_eq = result.x
print(f"Equilibrium prices: p1 = {p1_eq:.4f}, p2 = {p2_eq:.4f}")

# Check if the equilibrium prices fulfill the market clearing conditions
check_market_clearing(p1_eq, p2_eq, w, par)


Equilibrium prices: p1 = 0.9759, p2 = 1.4908
Labor Market Clearing Condition: -0.000005
Goods Market 1 Clearing Condition: -0.000016
Goods Market 2 Clearing Condition: 0.000008


In [ ]:
import numpy as np
from types import SimpleNamespace

# Given equilibrium prices
p1_eq = 0.9759
p2_eq = 1.4908
w = 1

# Parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.kappa = 0.1

# Function to compute firm's optimal labor and output
def firm_behavior(w, p, A, gamma):
    L_star = (p * A * gamma / w) ** (1 / (1 - gamma))
    Y_star = A * L_star ** gamma
    Pi_star = w * L_star * (1 - gamma) / gamma
    return L_star, Y_star, Pi_star

# Function to derive labor supply L_star
def derive_labor_supply(par, w, p1, p2, tau, T, Pi1, Pi2):
    l = sp.symbols('l')
    
    # Consumption functions in terms of labor supply l
    c1 = par.alpha * (w * l + T + Pi1 + Pi2) / p1
    c2 = (1 - par.alpha) * (w * l + T + Pi1 + Pi2) / (p2 + tau)
    
    # Utility function
    utility = sp.log(c1**par.alpha * c2**(1 - par.alpha)) - par.nu * (l**(1 + par.epsilon)) / (1 + par.epsilon)
    
    # First-order condition (FOC) for utility maximization
    foc = sp.diff(utility, l)
    
    # Solve the FOC for labor supply L_star
    L_star_solutions = sp.solve(foc, l)
    
    # Filter only real and positive solutions
    L_star = [sol.evalf() for sol in L_star_solutions if sol.is_real and sol > 0]
    
    return L_star[0] if L_star else None

# Function to compute consumer's optimal consumption
def consumer_behavior(w, p1, p2, tau, T, alpha, nu, epsilon, Pi1, Pi2):
    # Derive L_star
    L_star = derive_labor_supply(par, w, p1, p2, tau, T, Pi1, Pi2)
    
    # Consumption given L_star
    C1_star = alpha * (w * L_star + T + Pi1 + Pi2) / p1
    C2_star = (1 - alpha) * (w * L_star + T + Pi1 + Pi2) / (p2 + tau)
    
    return L_star, C1_star, C2_star

# Calculate values using the given equilibrium prices
L1_star, Y1_star, Pi1_star = firm_behavior(w, p1_eq, par.A, par.gamma)
L2_star, Y2_star, Pi2_star = firm_behavior(w, p2_eq, par.A, par.gamma)
L_star, C1_star, C2_star = consumer_behavior(w, p1_eq, p2_eq, par.tau, par.T, par.alpha, par.nu, par.epsilon, Pi1_star, Pi2_star)

# Display the results
print(f"L1_star = {L1_star:.6f}")
print(f"Y1_star = {Y1_star:.6f}")
print(f"L2_star = {L2_star:.6f}")
print(f"Y2_star = {Y2_star:.6f}")
print(f"C1_star = {C1_star:.6f}")
print(f"C2_star = {C2_star:.6f}")

  


L1_star = 0.238095
Y1_star = 0.487950
L2_star = 0.555621
Y2_star = 0.745400
C1_star = 0.487985
C2_star = 0.745365


# NY METODE


In [ ]:
import numpy as np
from types import SimpleNamespace

# Parameters

class Model:
    def __init__(self, par):

        par = self.par = SimpleNamespace()
        par.A = 1.0
        par.gamma = 0.5
        par.alpha = 0.3
        par.nu = 1.0
        par.epsilon = 2.0
        par.tau = 0.0
        par.T = 0.0
        par.kappa = 0.1
        
        self.w = 1  # Assuming wage is 1 as numeraire

    def optimal_behavior(self, l):
        c1 = self.c1(l)
        c2 = self.c2(l)
        utility = np.log(c1**self.par.alpha * c2**(1 - self.par.alpha)) - self.par.nu * (l**(1 + self.par.epsilon)) / (1 + self.par.epsilon)
        return utility

    def c1(self, l):
        return self.par.alpha * (self.w * l + self.par.T + self.pi1() + self.pi2()) / self.p1

    def c2(self, l):
        return (1 - self.par.alpha) * (self.w * l + self.par.T + self.pi1() + self.pi2()) / (self.p2 + self.par.tau)

    def pi1(self):
        return (1 - self.par.gamma) / self.par.gamma * self.w * ((self.p1 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma)))

    def pi2(self):
        return (1 - self.par.gamma) / self.par.gamma * self.w * ((self.p2 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma)))

    def labor_market(self):
        l1_star = (self.p1 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma))
        l2_star = (self.p2 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma))
        return l1_star + l2_star

    def good_market_1(self):
        return self.c1_star() - self.y1_star()

    def good_market_2(self):
        return self.c2_star() - self.y2_star()

    def y1_star(self):
        return self.par.A * (self.labor_market() / 2)**self.par.gamma

    def y2_star(self):
        return self.par.A * (self.labor_market() / 2)**self.par.gamma

    def c1_star(self):
        return self.c1(self.labor_market())

    def c2_star(self):
        return self.c2(self.labor_market())


# This one below

In [ ]:
    def utility_(self, l):
        c1 = self.c1(l)
        c2 = self.c2(l)
        return np.log(c1**self.par.alpha * c2**(1 - self.par.alpha)) - self.par.nu * (l**(1 + self.par.epsilon)) / (1 + self.par.epsilon)

    def optimize_l(self):
        """ Maximize utility for workers """
        
        # a. solve
        obj = lambda l: -self.utility_(l)
        res = optimize.minimize_scalar(obj, bounds=(0, 1), method='bounded')
        
        # b. save
        l_star = res.x
        c1_star = self.c1(l_star)
        c2_star = self.c2(l_star)
        
        return l_star, c1_star, c2_star

def firm_labor_demand(self):
        l1_star = (self.p1 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma))
        l2_star = (self.p2 * self.par.A * self.par.gamma / self.w)**(1 / (1 - self.par.gamma))
        return l1_star + l2_star

def firm_profit1(self):
        y1_star = self.par.A * (l1_star)**self.par.gamma
        return y1_star

def firm_profit2(self):
        y2_star = self.par.A * (l2_star)**self.par.gamma
        return y2_star

def evaluate_equilibrium(w,par,p=None,do_print=False):
    """ evaluate equilirium """
    
    
    # d. market clearing
    goods1_mkt_clearing = c1_star - y1star 
    labor_mkt_clearing = l_star = l1_star+l2_star
    
    if do_print:
        
        u_w = utility_w(c_w_star,l_w_star,par)
        print(f'workers      : c = {c_w_star:6.4f}, l = {l_w_star:6.4f}, u = {u_w:7.4f}')
        u_c = utility_c(c_c_star,l_c_star,par)
        print(f'capitalists  : c = {c_c_star:6.4f}, l = {l_c_star:6.4f}, u = {u_c:7.4f}')        
        print(f'goods market : {goods_mkt_clearing:.8f}')
        print(f'labor market : {labor_mkt_clearing:.8f}')
        
    else:
    
        return goods_mkt_clearing

SyntaxError: expected ':' (1034500516.py, line 20)

# This one above 

In [ ]:

import numpy as np
from scipy import optimize
from types import SimpleNamespace

class UtilityModel:
    def __init__(self, alpha, nu, epsilon, T, tau, w, A, gamma):
        self.par = SimpleNamespace(alpha=alpha, nu=nu, epsilon=epsilon, T=T, tau=tau, w=w, A=A, gamma=gamma, p1=p1, p2=p2)

    def firm_l_star(self, pj):
        return (pj * self.par.A * self.par.gamma / self.par.w) ** (1 / (1 - self.par.gamma))

    def firm_pi_star(self, pj):
        l_star = self.firm_l_star(pj)
        return (1 - self.par.gamma) / self.par.gamma * self.par.w * l_star

    def c1(self, l):
        pi1_star = self.firm_pi_star(self.par.p1)
        pi2_star = self.firm_pi_star(self.par.p2)
        return self.par.alpha * (self.par.w * l + self.par.T + pi1_star + pi2_star) / self.par.p1

    def c2(self, l):
        pi1_star = self.firm_pi_star(self.par.p1)
        pi2_star = self.firm_pi_star(self.par.p2)
        return (1 - self.par.alpha) * (self.par.w * l + self.par.T + pi1_star + pi2_star) / (self.par.p2 + self.par.tau)

    def utility_(self, l):
        c1 = self.c1(l)
        c2 = self.c2(l)
        return np.log(c1**self.par.alpha * c2**(1 - self.par.alpha)) - self.par.nu * (l**(1 + self.par.epsilon)) / (1 + self.par.epsilon)

    def optimize_l(self):
        obj = lambda l: -self.utility_(l)
        res = optimize.minimize_scalar(obj, bounds=(0, 1), method='bounded')
        l_star = res.x
        c1_star = self.c1(l_star)
        c2_star = self.c2(l_star)
        return l_star, c1_star, c2_star

    def firm_labor_demand(self, p1, p2):
        l1_star = (p1 * self.par.A * self.par.gamma / self.par.w)**(1 / (1 - self.par.gamma))
        l2_star = (p2 * self.par.A * self.par.gamma / self.par.w)**(1 / (1 - self.par.gamma))
        return l1_star, l2_star

    def firm_output(self, l_star):
        return self.par.A * (l_star)**self.par.gamma

def market_clearing_conditions(prices, model):
    p1, p2 = prices
    l1_star, l2_star = model.firm_labor_demand(p1, p2)
    y1_star = model.firm_output(l1_star)
    y2_star = model.firm_output(l2_star)
    
    l_star, c1_star, c2_star = model.optimize_l()

    goods1_mkt_clearing = c1_star - y1_star
    goods2_mkt_clearing = c2_star - y2_star
    labor_mkt_clearing = l_star - (l1_star + l2_star)

    return [goods1_mkt_clearing, goods2_mkt_clearing, labor_mkt_clearing]

def find_equilibrium(par):
    model = UtilityModel(par.alpha, par.nu, par.epsilon, par.T, par.tau, par.w, par.A, par.gamma)
    initial_guess = [par.p1, par.p2]
    result = optimize.root(market_clearing_conditions, initial_guess, args=(model,), method='hybr')

    if result.success:
        p1_star, p2_star = result.x
        l_star, c1_star, c2_star = model.optimize_l()
        print(f'Equilibrium prices: p1* = {p1_star:.4f}, p2* = {p2_star:.4f}')
        print(f'Optimal labor: l* = {l_star:.4f}')
        print(f'Optimal consumption: c1* = {c1_star:.4f}, c2* = {c2_star:.4f}')
    else:
        print("Equilibrium not found")



# Example usage:
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.w = 1.0  # Assuming wage is 1 as numeraire
find_equilibrium(par)

AttributeError: 'types.SimpleNamespace' object has no attribute 'p1'

**Question 2:** Find the equilibrium prices $p_1$ and $p_2$.<br>
*Hint: you can use Walras' law to only check 2 of the market clearings*

In [ ]:
# write your answer here

Assume the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

Here $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Question 3:** What values of $\tau$ and (implied) $T$ should the government choose to maximize $SWF$?

In [ ]:
# write your answer here

## 2. <a id='toc2_'></a>[Problem 2: Career choice model](#toc0_)

Consider a graduate $i$ making a choice between entering $J$ different career tracks. <br>
Entering career $j$ yields utility $u^k_{ij}$. This value is unknown to the graduate ex ante, but will ex post be: <br>
$$
    u_{i,j}^k = v_{j} + \epsilon_{i,j}^k
$$

They know that $\epsilon^k_{i,j}\sim \mathcal{N}(0,\sigma^2)$, but they do not observe $\epsilon^k_{i,j}$ before making their career choice. <br>

Consider the concrete case of $J=3$ with:
$$
\begin{align*}
    v_{1} &= 1 \\
    v_{2} &= 2 \\
    v_{3} &= 3
\end{align*}
$$

If the graduates know the values of $v_j$ and the distribution of $\epsilon_{i,j}^k$, they can calculate the expected utility of each career track using simulation: <br>
$$
    \mathbb{E}\left[ u^k_{i,j}\vert v_j \right] \approx v_j + \frac{1}{K}\sum_{k=1}^K \epsilon_{i,j}^k
$$

In [ ]:
par = SimpleNamespace()
par.J = 3
par.N = 10
par.K = 10000

par.F = np.arange(1,par.N+1)
par.sigma = 2

par.v = np.array([1,2,3])
par.c = 1

**Question 1:** Simulate and calculate expected utility and the average realised utility for $K=10000$ draws, for each career choice $j$.


In [ ]:
# write your answer here

Now consider a new scenario: Imagine that the graduate does not know $v_j$. The *only* prior information they have on the value of each job, comes from their $F_{i}$ friends that work in each career $j$. After talking with them, they know the average utility of their friends (which includes their friends' noise term), giving them the prior expecation: <br>
$$
\tilde{u}^k_{i,j}\left( F_{i}\right) = \frac{1}{F_{i}}\sum_{f=1}^{F_{i}} \left(v_{j} + \epsilon^k_{f,j}\right), \; \epsilon^k_{f,j}\sim \mathcal{N}(0,\sigma^2)
$$
For ease of notation consider that each graduate have $F_{i}=i$ friends in each career. <br>

For $K$ times do the following: <br>
1. For each person $i$ draw $J\cdot F_i$ values of $\epsilon_{f,j}^{k}$, and calculate the prior expected utility of each career track, $\tilde{u}^k_{i,j}\left( F_{i}\right)$. <br>
Also draw their own $J$ noise terms, $\epsilon_{i,j}^k$
1. Each person $i$ chooses the career track with the highest expected utility: $$j_i^{k*}= \arg\max_{j\in{1,2\dots,J}}\left\{ \tilde{u}^k_{i,j}\left( F_{i}\right)\right\} $$
1. Store the chosen careers: $j_i^{k*}$, the prior expectation of the value of their chosen career: $\tilde{u}^k_{i,j=j_i^{k*}}\left( F_{i}\right)$, and the realized value of their chosen career track: $u^k_{i,j=j_i^{k*}}=v_{j=j_i^{k*}}+\epsilon_{i,j=j_i^{k*}}^k$.

Chosen values will be: <br>
$i\in\left\{1,2\dots,N\right\}, N=10$ <br>
$F_i = i$<br>
So there are 10 graduates. The first has 1 friend in each career, the second has 2 friends, ... the tenth has 10 friends.

**Question 2:** Simulate and visualize: For each type of graduate, $i$, the share of graduates choosing each career, the average subjective expected utility of the graduates, and the average ex post realized utility given their choice. <br>
That is, calculate and visualize: <br>
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \mathbb{I}\left\{ j=j_i^{k*} \right\}  \;\forall j\in\left\{1,2,\dots,J\right\}
\end{align*}
$$
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \tilde{u}^k_{ij=j_i^{k*}}\left( F_{i}\right)
\end{align*}
$$
And 
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} u^k_{ij=j_i^{k*}} 
\end{align*}
$$
For each graduate $i$.

In [ ]:
# Write your answer here 

After a year of working in their career, the graduates learn $u^k_{ij}$ for their chosen job $j_i^{k*}$ perfectly. <br>
The can switch to one of the two remaining careers, for which they have the same prior as before, but it will now include a switching cost of $c$ which is known.
Their new priors can be written as: 
$$
\tilde{u}^{k,2}_{ij}\left( F_{i}\right) = \begin{cases}
            \tilde{u}^k_{ij}\left( F_{i}\right)-c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

We will set $c=1$.

Their realized utility will be: <br>
$$
u^{k,2}_{ij}= \begin{cases}
            u_{ij}^k -c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

**Question 3:** Following the same approach as in question 2, find the new optimal career choice for each $i$, $k$. Then for each $i$, calculate the average subjective expected utility from their new optimal career choice, and the ex post realized utility of that career. Also, for each $i$, calculate the share of graduates that chooses to switch careers, conditional on which career they chose in the first year. <br>

In [ ]:
# write your answer here

## 3. <a id='toc3_'></a>[Problem 3: Barycentric interpolation](#toc0_)

**Problem:** We have a set of random points in the unit square,

$$
\mathcal{X} = \{(x_1,x_2)\,|\,x_1\sim\mathcal{U}(0,1),x_2\sim\mathcal{U}(0,1)\}.
$$

For these points, we know the value of some function $f(x_1,x_2)$,

$$
\mathcal{F} = \{f(x_1,x_2) \,|\, (x_1,x_2) \in \mathcal{X}\}.
$$

Now we want to approximate the value $f(y_1,y_2)$ for some  $y=(y_1,y_2)$, where $y_1\sim\mathcal{U}(0,1)$ and $y_2\sim\mathcal{U}(0,1)$.

**Building block I**

For an arbitrary triangle $ABC$ and a point $y$, define the so-called barycentric coordinates as:

$$
\begin{align*}
  r^{ABC}_1 &= \frac{(B_2-C_2)(y_1-C_1) + (C_1-B_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_2 &= \frac{(C_2-A_2)(y_1-C_1) + (A_1-C_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_3 &= 1 - r_1 - r_2.
\end{align*}
$$

If $r^{ABC}_1 \in [0,1]$, $r^{ABC}_2 \in [0,1]$, and $r^{ABC}_3 \in [0,1]$, then the point is inside the triangle.

We always have $y = r^{ABC}_1 A + r^{ABC}_2 B + r^{ABC}_3 C$.

**Building block II**

Define the following points:

$$
\begin{align*}
A&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}>y_{2}\\
B&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}<y_{2}\\
C&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}<y_{2}\\
D&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}>y_{2}.
\end{align*}
$$

**Algorithm:**

1. Compute $A$, $B$, $C$, and $D$. If not possible return `NaN`.
1. If $y$ is inside the triangle $ABC$ return $r^{ABC}_1 f(A) + r^{ABC}_2 f(B) + r^{ABC}_3 f(C)$.
1. If $y$ is inside the triangle $CDA$ return $r^{CDA}_1 f(C) + r^{CDA}_2 f(D) + r^{CDA}_3 f(A)$.
1. Return `NaN`.



**Sample:**

In [ ]:
rng = np.random.default_rng(2024)

X = rng.uniform(size=(50,2))
y = rng.uniform(size=(2,))


**Questions 1:** Find $A$, $B$, $C$ and $D$. Illustrate these together with $X$, $y$ and the triangles $ABC$ and $CDA$.

In [ ]:
# write your answer here

**Question 2:** Compute the barycentric coordinates of the point $y$ with respect to the triangles $ABC$ and $CDA$. Which triangle is $y$ located inside?

In [ ]:
# write your answer here

Now consider the function:
$$
f(x_1,x_2) = x_1 \cdot x_2
$$

In [ ]:
f = lambda x: x[0]*x[1]
F = np.array([f(x) for x in X])

**Question 3:** Compute the approximation of $f(y)$ using the full algorithm. Compare with the true value.

In [ ]:
# write your answer here

**Question 4:** Repeat question 3 for all points in the set $Y$.

In [ ]:
Y = [(0.2,0.2),(0.8,0.2),(0.8,0.8),(0.8,0.2),(0.5,0.5)]

In [ ]:
# write your answer here